## 市場ポテンシャル分析 ― 国籍別の量的変化と消費水準から捉える市場構造の再評価 ―

### 目的とアジェンダ

#### 分析の目的

訪日客の国籍構成比の変化および1人あたりの消費額（消費単価）の水準に着目し、欧米豪市場が日本のインバウンド市場においてどのように位置づけられるのかを整理・評価する。

その結果をもとに、従来の東アジア中心の市場戦略を再検討し、今後の市場戦略の方向性を検討するためのインサイトを得ることを目的とする。

#### 主要な分析ステップ

1.  訪日客数の推移と国籍構成の変化（**量的視点**）
2.  1人あたりの消費額の国籍別比較（**質的視点**）
3.  市場ポテンシャル・マッピング（**4象限分析**）による各国の市場ポジションの把握


### 環境構築とデータ準備

データ読込と市場ポテンシャルデータの生成（客数と消費額の統合）

In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display

# 表示設定
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# ノートブックが 'notebooks/' ディレクトリにあることを前提とする
DATA_PATH = '../data/'

# データ読込
try:
    # 必須ファイル
    df_visiter = pd.read_csv(os.path.join(DATA_PATH, 'inbound_visiter.csv'))
    df_spend = pd.read_csv(os.path.join(DATA_PATH, 'inbound_spending.csv'))
    
except FileNotFoundError as e:
    print(f"エラー: {e.filename} が見つかりません。データファイルを 'data/' フォルダに配置してください。")
    raise

# データ統合・集計

# 国・地域名の統一と集計
df_visiter['Country/Area'] = df_visiter['Country/Area'].replace('全国籍･地域', 'All Countries')

# 訪日客数の年間集計
df_yearly_visiter = df_visiter.groupby(['Year', 'Country/Area'])['Visitor_Numeric'].sum().reset_index()
df_yearly_visiter.rename(columns={'Visitor_Numeric': 'Annual_Visitors', 'Country/Area': 'country', 'Year': 'year'}, inplace=True)
# ↑ 'Year' (大文字) を 'year' (小文字) にリネームを追加

# 一人あたり消費額の年間集計（'全体'のconsumption_unitを使用）
df_avg_spend_yearly = df_spend[
    (df_spend['expense_items'].str.contains(r'^全体', regex=True, na=False)) & 
    (df_spend['details'] == 'all') & 
    (df_spend['country'] != '全国籍･地域') # 全体集計は除く
].copy()

# データを年次・国別で集計 (平均値)
df_avg_spend_yearly = df_avg_spend_yearly.groupby(['year', 'country'])['consumption_unit'].mean().reset_index()
df_avg_spend_yearly.rename(columns={'consumption_unit': 'Avg_Total_Spend'}, inplace=True)

# データの結合
df_market_potential_yearly = pd.merge(
    df_yearly_visiter,
    df_avg_spend_yearly,
    on=['year', 'country'],
    how='inner'
)

# 市場ポテンシャル総額の計算
df_market_potential_yearly['Market_Potential_Total'] = \
    df_market_potential_yearly['Annual_Visitors'] * df_market_potential_yearly['Avg_Total_Spend']

# 地域グループの定義（分析の課題意識に基づく）
def define_region(country):
    # 欧米豪グループ
    if country in ['米国', 'カナダ', '英国', 'フランス', 'ドイツ', 'イタリア', 'スペイン', 'オーストラリア', 'ニュージーランド']:
        return '欧米豪'
    
    # 東アジアグループ
    elif country in ['韓国', '中国', '台湾', '香港']:
        return '東アジア'
        
    # 東南アジアグループ
    elif country in ['タイ', 'シンガポール', 'マレーシア', 'インドネシア', 'フィリピン', 'ベトナム']:
        return '東南アジア'
        
    # その他
    else:
        return 'その他'

df_market_potential_yearly['Region_Group'] = df_market_potential_yearly['country'].apply(define_region)

# 最新年と2019年を定義
LATEST_YEAR = df_market_potential_yearly['year'].max()
PRE_COVID_YEAR = 2019

print(f"最新年: {LATEST_YEAR}, 2019年データ数: {len(df_market_potential_yearly[df_market_potential_yearly['year'] == PRE_COVID_YEAR])}")
print("--- 統合データ（抜粋） ---")
display(df_market_potential_yearly.head(), df_market_potential_yearly.tail())

最新年: 2024, 2019年データ数: 19
--- 統合データ（抜粋） ---


,year,country,Annual_Visitors,Avg_Total_Spend,Market_Potential_Total,Region_Group
0,2019,その他,1559472,224168.50,3.495845e+11,その他
1,2019,イタリア,132521,221564.75,2.936198e+10,欧米豪
2,2019,インド,75558,173757.25,1.312875e+10,その他
3,2019,インドネシア,339133,136201.75,4.619051e+10,東南アジア
4,2019,オーストラリア,571227,259698.00,1.483465e+11,欧米豪


,year,country,Annual_Visitors,Avg_Total_Spend,Market_Potential_Total,Region_Group
71,2024,台湾,5867101,184120.00,1.080251e+12,東アジア
72,2024,米国,2537002,343164.75,8.706097e+11,欧米豪
73,2024,英国,387205,391242.75,1.514911e+11,欧米豪
74,2024,韓国,8452682,104046.00,8.794678e+11,東アジア
75,2024,香港,2637761,245950.75,6.487593e+11,東アジア


### 1.  訪日客数の量的変化の分析

パンデミック後の訪日市場における構造変化を地域グループごとの構成比および訪日客数の2019年比回復状況を通じて定量的に検証する。


In [ ]:
# 国籍構成比の年次変化と2019年との訪日客数差分

# 地域別訪問者数を計算（構成比用）
df_region_visitors = df_market_potential_yearly.groupby(['year', 'Region_Group'])['Annual_Visitors'].sum().reset_index()

# 構成比の計算
df_region_composition = df_region_visitors.pivot_table(
    index='year', 
    columns='Region_Group', 
    values='Annual_Visitors',
    aggfunc='sum'
)
# 構成比に変換
df_region_composition_ratio = df_region_composition.div(df_region_composition.sum(axis=1), axis=0) * 100

# ----------------------------------------------------------------------
# 地域グループ別構成比の推移 (折れ線グラフ)
# ----------------------------------------------------------------------

# 縦持ちデータフレームを作成
df_line_chart_data = df_region_composition_ratio.reset_index().melt(
    id_vars=['year'], 
    value_vars=df_region_composition_ratio.columns, 
    var_name='Region_Group', 
    value_name='Composition_Ratio'
)

# グラフ化：構成比の推移 
fig_composition = px.line(
    df_line_chart_data, 
    x='year',
    y='Composition_Ratio', 
    color='Region_Group', 
    title='訪日客数の地域グループ別構成比の推移 (全体)',
    labels={'year': '年', 'Composition_Ratio': '構成比 (%)', 'Region_Group': '地域グループ'},
    color_discrete_map={
        '欧米豪': '#1f77b4', 
        '東アジア': '#ff7f0e',
        '東南アジア': '#d62728',
        'その他': '#2ca02c'
    }
)

fig_composition.update_traces(mode='lines+markers', marker=dict(size=8))
fig_composition.update_xaxes(tick0=df_line_chart_data['year'].min(), dtick=1, tickformat='d') 
fig_composition.update_layout(height=400, width=900)
fig_composition.show()

# ----------------------------------------------------------------------
# 2019年と最新年の訪問者数差分表
# ----------------------------------------------------------------------

for group_name in df_market_potential_yearly['Region_Group'].unique():
    
    # グループ名が「欧米豪」の場合のみ、以降の処理を実行する
    if group_name != '欧米豪':
        continue 
        
    print(f"{group_name} グループの差分表 (最新年 vs 2019年)")
    
    # 現在のグループのデータに絞り込み
    df_group = df_market_potential_yearly[df_market_potential_yearly['Region_Group'] == group_name].copy()

    # 2019年と最新年(LATEST_YEAR)のデータをピボットテーブルで抽出
    df_pivot = df_group.pivot_table(
        index='country', 
        columns='year', 
        values='Annual_Visitors',
        aggfunc='sum'
    )
    
    # 結合後のデータに2019年または最新年のデータが存在しない国を除外
    if PRE_COVID_YEAR not in df_pivot.columns or LATEST_YEAR not in df_pivot.columns:
        print(f"注: {group_name} グループには、{PRE_COVID_YEAR}年または {LATEST_YEAR}年のデータが不足している国があります。")
        continue

    # 差分を計算: 最新年 - 2019年
    df_diff = pd.DataFrame({
        '2019年_訪問者数': df_pivot[PRE_COVID_YEAR],
        f'{LATEST_YEAR}年_訪問者数': df_pivot[LATEST_YEAR],
    })
    df_diff['差分 (最新年 - 2019年)'] = df_diff[f'{LATEST_YEAR}年_訪問者数'] - df_diff['2019年_訪問者数']
    df_diff['回復率 (最新年/2019年)'] = df_diff[f'{LATEST_YEAR}年_訪問者数'] / df_diff['2019年_訪問者数']

    # 差分が大きい順にソートし、表示形式を整える
    df_diff_sorted = df_diff.sort_values(by='差分 (最新年 - 2019年)', ascending=False)
    
    # スタイリングの適用
    df_diff_styled = df_diff_sorted.style.format({
        '2019年_訪問者数': '{:,d}',
        f'{LATEST_YEAR}年_訪問者数': '{:,d}',
        '差分 (最新年 - 2019年)': '{:+,d}',
        '回復率 (最新年/2019年)': '{:.1%}'
    })

    display(df_diff_styled)

欧米豪 グループの差分表 (最新年 vs 2019年)


,2019年_訪問者数,2024年_訪問者数,差分 (最新年 - 2019年),回復率 (最新年/2019年)
country,,,,
米国,"1,429,036","2,537,002","+1,107,966",177.5%
オーストラリア,"571,227","885,303","+314,076",155.0%
カナダ,"339,319","557,338","+218,019",164.3%
ドイツ,"168,214","283,427","+115,213",168.5%
イタリア,"132,521","208,973","+76,452",157.7%
フランス,"270,660","342,207","+71,547",126.4%
スペイン,"114,252","171,175","+56,923",149.8%
英国,"343,122","387,205","+44,083",112.8%


欧米豪市場の構成比は、2019年の11.9%から2024年には16.0%へと拡大しており、コロナ前と比較して相対的な存在感を高めている。  
差分表では国別に回復度合いの差が見られることから、欧米豪市場は一括りではなく、国ごとの特性を踏まえた戦略設計を要することが示唆される。

### 2.  1人あたりの消費額の国籍別比較

最新年の各国の1人あたりの消費額（消費単価）を確認し、市場の「質」を評価する。


In [ ]:
# 最新年の国籍別の1人あたりの消費額ランキング

df_latest_spend = df_market_potential_yearly[
    (df_market_potential_yearly['year'] == LATEST_YEAR) & 
    (df_market_potential_yearly['country'] != 'All Countries')
].sort_values('Avg_Total_Spend', ascending=False)

fig_spend_rank = px.bar(
    df_latest_spend,
    x='Avg_Total_Spend',
    y='country',
    orientation='h',
    color='Region_Group',
    title=f'{LATEST_YEAR}年 国籍別 1人あたりの消費額（消費単価）ランキング',
    labels={'Avg_Total_Spend': '1人あたりの消費額 (円)', 'country': '国・地域'},
)
fig_spend_rank.update_layout(yaxis={'categoryorder':'total ascending'}, height=500)
fig_spend_rank.show()

一人あたり消費額ランキングの上位には欧米豪の国が多く含まれており、同地域が相対的に高い消費水準を持つ市場であることが確認できる。  
一方で、国ごとに消費額の水準には差が見られるため、欧米豪市場を一様に捉えるのではなく、国別の消費特性にも留意して整理していく視点が有用であると思われる。

### 5. 市場ポテンシャル・マッピング（4象限分析）

**「量」**（訪日客数）と **「質」**（1人あたりの消費額）を統合し、各国・地域の戦略的な位置づけを評価する。


In [ ]:
# ----------------------------------------------------------------------
# 最新年のデータ抽出
# ----------------------------------------------------------------------
df_map = df_market_potential_yearly[
    (df_market_potential_yearly['year'] == LATEST_YEAR) &
    (df_market_potential_yearly['country'] != 'All Countries')
].copy()

# ----------------------------------------------------------------------
# 中央値を「対数空間」で計算 → 元スケールへ戻す
# ----------------------------------------------------------------------
df_map['log_visitors'] = np.log(df_map['Annual_Visitors'])
df_map['log_spend'] = np.log(df_map['Avg_Total_Spend'])

median_log_visitor = df_map['log_visitors'].median()
median_log_spend = df_map['log_spend'].median()

median_visitor = float(np.exp(median_log_visitor))
median_spend = float(np.exp(median_log_spend))

# ----------------------------------------------------------------------
# 市場ポテンシャル・マッピング
#   x：訪日客数（量・対数）
#   y：消費額（質・対数）
# ----------------------------------------------------------------------
fig_scatter = px.scatter(
    df_map,
    x='Annual_Visitors',           # ← 量をx軸
    y='Avg_Total_Spend',           # ← 質をy軸
    size='Market_Potential_Total',
    text='country',
    hover_name='country',
    color='Region_Group',
    log_x=True,
    log_y=True,
    title=f'{LATEST_YEAR}年 市場ポテンシャル・マッピング (量×質)',
    labels={
        'Annual_Visitors': '年間訪日客数（量）[対数]',
        'Avg_Total_Spend': '一人あたり消費額（質）[対数]',
        'Market_Potential_Total': '総市場ポテンシャル（円）'
    }
)

# ----------------------------------------------------------------------
# 4象限ライン（対数中央値を元スケールで描画）
# ----------------------------------------------------------------------
fig_scatter.add_vline(
    x=median_visitor,
    line_dash="dash",
    line_color="red",
    annotation_text=f"訪日客数（対数中央値） ({median_visitor:,.0f}人)",
    annotation_position="top"
)

fig_scatter.add_hline(
    y=median_spend,
    line_dash="dash",
    line_color="blue",
    annotation_text=f"消費額（対数中央値） ({median_spend:,.0f}円)"
)

# ----------------------------------------------------------------------
# 象限ラベル（相対位置で配置）
# ----------------------------------------------------------------------

# HH:(右上)
fig_scatter.add_annotation(
    xref='paper', yref='paper', # 参照をpaperモードに設定
    x=0.90, y=0.90,             # グラフの(75%, 75%)の位置 (右上象限)
    text="HH: 戦略的中核市場（量×質）",
    showarrow=False, 
    font=dict(size=11, color="black"),
    xshift=0, yshift=0
)

# HL:(右下)
fig_scatter.add_annotation(
    xref='paper', yref='paper',
    x=0.80, y=0.25,             # グラフの(75%, 25%)の位置 (右下象限)
    text="HL: 量主導市場（単価改善余地）",
    showarrow=False, 
    font=dict(size=11, color="black"),
    xshift=0, yshift=0
)

# LH:(左上)
fig_scatter.add_annotation(
    xref='paper', yref='paper',
    x=0.05, y=0.95,             # グラフの(25%, 75%)の位置 (左上象限)
    text="LH: 高付加価値市場（質重視）",
    showarrow=False, 
    font=dict(size=11, color="black"),
    xshift=0, yshift=0
)

# LL:(左下)
fig_scatter.add_annotation(
    xref='paper', yref='paper',
    x=0.15, y=0.25,             # グラフの(25%, 25%)の位置 (左下象限)
    text="LL: 育成市場（探索・維持）",
    showarrow=False, 
    font=dict(size=11, color="black"),
    xshift=0, yshift=0
)
# ----------------------------------------------------------------------
# 最終調整と表示
# ----------------------------------------------------------------------

fig_scatter.update_traces(
    textposition='bottom center',
    marker=dict(opacity=0.8, line=dict(width=1, color='Black'))
)
fig_scatter.update_layout(height=650)
fig_scatter.show()

### 6. 結論とインサイト

1. **市場構成の変化に関する示唆**  
訪日客数構成比の推移から、欧米豪市場は2019年と比較して直近年では相対的に高い水準に位置しており、年ごとの変動も見られることから中長期的な傾向としては慎重に捉える必要があるものの、日本のインバウンド市場における存在感が変化してきていることが確認できる。  
（**量的な存在感の上昇**）  

2. **市場ポジションの分布と戦略的示唆**  
市場ポテンシャル・マッピングの結果、米国およびオーストラリアは訪日客数と1人あたりの消費額の両方が高い **HH 象限** に位置していることが確認された。  
また、英国、フランス、ドイツなどの欧州諸国は、1人あたりの消費額が高い **LH 象限** に分布している。  
したがって、欧米豪市場は国ごとに市場規模に差があるものの、**1人あたりの消費額の水準が相対的に高い国を多く含む市場群である**ことが明らかとなった。  
（**質的な優位性**）

3. **結論**  
以上を踏まえると、従来の東アジア中心の市場戦略を前提とした市場評価を見直し、1人あたりの消費額の水準が高い欧米豪市場について、国ごとの市場特性を踏まえながら重要な検討対象として捉え直すことは、今後の日本のインバウンド市場戦略を検討するうえで有効な示唆を与えると考えられる。
